In [65]:
import pandas as pd
import numpy as np
import random
import os

In [66]:
def duplicar_random(file_path, n):
    lines = open(F"{file_path}").readlines()
    lista_archivos = []
    for i in range(n):
        random.shuffle(lines)
        with open(F"../scripts/instancias/random/random-{i}.txt", 'w') as f:
            f.writelines(lines)
        lista_archivos.append(F"../scripts/instancias/random/random-{i}.txt")
    with open(F"instancias/random/indice.txt", 'w') as f:
        print("Indice archivos",file=f)
        for j in range(n):
            print(lista_archivos[j], file=f)

## Muchos vs pocos archivos

In [11]:
def juntar_en_un_archivo(source_directory_path,target_file_path):
    df_indice = pd.read_csv(source_directory_path + "/indice.txt")
    with open(target_file_path,'w') as target:
        for source_path in df_indice.stack().to_list():
            with open(source_path, 'r') as source:
                target.write(source.read())

In [63]:
def separar_en_archivos(source_file_path, n, target_directory_path):
    source_list = pd.read_csv(source_file_path).stack().to_list()
    output_line_count = len(source_list)//n
    current_line = 0
    lista_indice = []
    for i in range(n):
        target_filepath = target_directory_path +"/separado-"+str(i)+".txt"
        lista_indice.append(target_filepath)
        with open(target_filepath, 'w') as target:
            for j in range(output_line_count):
                try:
                    print(source_list[current_line], file=target)
                except IndexError:
                    print(F"fallo en la linea {current_line}/{len(source_list)} en el archivo Nro {i} durante la iteracion Nro {j}")
                    raise IndexError("list index out of range")
                current_line += 1
            #En el ultimo archivo metemos las lineas "que faltaban" por el redondeo
            if(i == n-1):
                while(current_line < len(source_list)):
                    print(source_list[current_line],file = target)
                    current_line += 1
    with open(target_directory_path+"/indice.txt",'w') as indice:
        print("Indice directorios",file=indice)
        for path in lista_indice:
            print(path,file=indice)


In [3]:
duplicar_random("../data/corpus", 100)

In [12]:
juntar_en_un_archivo(source_directory_path="instancias/random",target_file_path="instancias/separados/fuente.txt")

In [64]:
cant_archivos = [2,4,8,16,32,64,128,256,512,1024]
with open("instancias/separados/indice_directorios.txt", 'w') as indice:
    print("Indice directorios",file=indice)
    for cantidad in cant_archivos:
        path = F"instancias/separados/{cantidad}_archivos"
        print(path,file=indice)
        try: 
            os.mkdir(path)
        except FileExistsError: 
            pass
        separar_en_archivos("instancias/separados/fuente.txt",cantidad,path)    